In [ ]:
import numpy as np
from math import exp, sqrt
import time
import numba



@numba.jit(nopython=True)
def payoff_asian_call(paths: np.ndarray, K: float, dt: float) -> np.ndarray:
    """
     Calculates the payoff for an Asian call option

    :param paths: 2D array containing asset price paths
    :param K: Strike price of the option
    :param dt: Time step

    :return: Payoff of each path
    """

    # Vectorized trapezoidal rule for integrating the path.
    integrated_sum = 0.5 * (paths[:, 0] + paths[:, -1]) + np.sum(paths[:, 1:-1], axis=1)

    # The time-averaged price S_bar
    S_bar = integrated_sum * dt

    # Asian call payoff
    payoffs = np.maximum(S_bar - K, 0.0)
    return payoffs


@numba.jit(nopython=True)
def srk_scheme(S0: float, dW: np.ndarray, dt: float, r: float, sigma: float) -> np.ndarray:
    """
    Simulates asset price paths using a two-stage Runge-Kutta scheme

    :param S0: Initial asset price
    :param dW: 2d array of Wiener Increments
    :param dt: Time step
    :param r: Risk free rate
    :param sigma: Volatility parameter

    :return: Simulated paths
    """

    num_paths, num_steps = dW.shape
    paths = np.zeros((num_paths, num_steps + 1))
    paths[:, 0] = S0
    sqrt_dt = sqrt(dt)

    for j in range(num_steps):
        S_n = paths[:, j]
        delta_W = dW[:, j]

        # Stage 1 (Predictor) Euler Step to calculate intermediate value is calculated.
        H_1 = S_n + r * S_n * dt + sigma * S_n * sqrt_dt

        # Stage 2 (Corrector) combines the Euler step with a derivative-free approximation of the Milstein correction term
        paths[:, j+1] = S_n + r * S_n * dt + sigma * S_n * delta_W + \
                        (sigma / (2.0 * sqrt_dt)) * (sigma * H_1 - sigma * S_n) * (delta_W**2 - dt)

        # Ensure positivity
        paths[:, j+1] = np.maximum(paths[:, j+1], 1e-12)

    return paths


@numba.jit(nopython=True)
def exact_gbm(S0: float, dW: np.ndarray, dt: float, r: float, sigma: float) -> np.ndarray:
    """
    Calculates the exact solution for geometric Brownian motion paths

    :param S0: Initial asset price
    :param dW: 2d array of Wiener Increments
    :param dt: Time step
    :param r: Risk free rate
    :param sigma: Volatility parameter

    :return: Simulated paths
    """
    num_paths, num_steps = dW.shape
    paths = np.zeros((num_paths, num_steps + 1))
    paths[:, 0] = S0

    # Build cumulative Brownian motion manually for numba
    W = np.zeros((num_paths, num_steps))

    for i in range(num_paths):
        W[i, 0] = dW[i, 0]
        for j in range(1, num_steps):
            W[i, j] = W[i, j-1] + dW[i, j]

    # Apply the exact solution formula at each time step.
    for j in range(num_steps):
        t = (j + 1) * dt
        for i in range(num_paths):
            paths[i, j+1] = S0 * exp((r - 0.5 * sigma**2) * t + sigma * W[i, j])

    return paths

@numba.jit(nopython=True)
def mlmc_sample_level_asian(path_simulator, S0:float, K:float, r:float, sigma:float, T:float, l:float, N_l:int, rng:int, M:float
                            ):
    """
    Generic MLMC level sampler for Asian options

    :param path_simulator: Function for path simulation
    :param S0: Initial asset price
    :param K: Strike price
    :param r: Risk free rate
    :param sigma: Volatility parameter
    :param T: Time step
    :param l: Current level index
    :param N_l: Number of samples to generate
    :param rng: Random number generator
    :param M: The refinement factor between levels

    :return: The samples of the estimator and the computational cost for this level
    """

    n_fine = M**l
    dt_f = T / n_fine

    # Cost is proportional to the number of steps on the fine grid
    cost = n_fine

    # Generate Wiener increments
    dW_f = rng.normal(0.0, sqrt(dt_f), size=(N_l, n_fine))

    if l == 0:
        # Level 0 We only need the coarse path
        paths_f = path_simulator(S0, dW_f, dt_f, r, sigma)
        Y = exp(-r * T) * payoff_asian_call(paths_f, K, dt_f)
    else:

        # The coarse Wiener increments are created by summing the fine ones
        dW_c = dW_f.reshape(N_l, n_fine // M, M).sum(axis=2)
        dt_c = T / (n_fine // M)

        # Simulate both fine and coarse paths using the coupled increments
        paths_f = path_simulator(S0, dW_f, dt_f, r, sigma)
        paths_c = path_simulator(S0, dW_c, dt_c, r, sigma)

        # Calculate payoffs for both levels
        payoff_f = payoff_asian_call(paths_f, K, dt_f)
        payoff_c = payoff_asian_call(paths_c, K, dt_c)

        # The estimator for this level is the discounted difference in payoffs
        Y = exp(-r * T) * (payoff_f - payoff_c)

    return Y, cost

def mlmc_adaptive_asian(scheme_name, path_simulator, S0:float, K:float, r:float, sigma:float, T:float, eps:float=0.01, seed:int=42, M:int=4):
    """
    Generic adaptive MLMC pricer for Asian options

    :param scheme_name: Name of the scheme
    :param path_simulator: Function for path simulation
    :param S0: Initial asset price
    :param K: Strike price
    :param r: Risk free rate
    :param sigma: Volatility parameter
    :param T: Time step
    :param eps: Target Root Mean Square Error
    :param seed: RNG seed
    :param M: The refinement factor between levels

    """
    print(f"\n=== Adaptive MLMC for Asian Option ({scheme_name} Scheme, M={M}) ===")
    rng = np.random.default_rng(seed)
    L_max = 10
    N0 = 2000

    means = np.zeros(L_max + 1)
    vars = np.zeros(L_max + 1)
    Nl = np.zeros(L_max + 1, dtype=int)
    costs = np.zeros(L_max + 1)

    L = 2
    # Start with a few levels to get initial estimates of variance and cost
    for l in range(L + 1):
        Nl[l] = max(100, N0 // (M**l))
        Y, cost = mlmc_sample_level_asian(path_simulator, S0, K, r, sigma, T, l, Nl[l], rng, M)
        means[l], vars[l], costs[l] = np.mean(Y), np.var(Y, ddof=1), cost
        print(f"Level {l}: N={Nl[l]}, mean={means[l]:+.4e}, var={vars[l]:.2e}")

    # Iteratively refine the estimates and sample allocation
    for iteration in range(15):
        total_var = sum(vars[l] / Nl[l] for l in range(L + 1))

        # Check for convergence: exit if variance is below the target
        if total_var < eps**2:
            print(f"Converged: total variance {total_var:.3e} < target {eps**2:.3e}")
            break

        # Check bias condition, add a new level if the bias
        if L < L_max and abs(means[L]) > eps / sqrt(2):
            L += 1
            Nl[L] = max(100, N0 // (M**L))
            Y, cost = mlmc_sample_level_asian(path_simulator, S0, K, r, sigma, T, L, Nl[L], rng, M)
            means[L], vars[L], costs[L] = np.mean(Y), np.var(Y, ddof=1), cost
            print(f"Added level {L}: N={Nl[L]}, mean={means[L]:+.4e}, var={vars[L]:.2e}")
            # Restart loop to re-evaluate optimal N with the new level
            continue

        # Calculate the optimal number of samples per level based on current estimates
        N_opt = np.ceil(2 * eps**-2 * np.sqrt(vars[:L+1] / costs[:L+1]) * sum(np.sqrt(vars[:L+1] * costs[:L+1])))

        # Generate additional samples for each level to meet the new optimal N_l
        for l in range(L + 1):
            extra = max(0, int(N_opt[l] - Nl[l]))
            if extra > 0:
                Y_ex, _ = mlmc_sample_level_asian(path_simulator, S0, K, r, sigma, T, l, extra, rng, M)
                total_N = Nl[l] + extra

                # Update mean and variance using stable formulas
                new_mean = (Nl[l] * means[l] + np.sum(Y_ex)) / total_N
                if Nl[l] > 1 and extra > 1:
                    vars[l] = ((Nl[l] - 1) * vars[l] + (extra - 1) * np.var(Y_ex, ddof=1) +
                               Nl[l] * (means[l] - new_mean)**2 +
                               extra * (np.mean(Y_ex) - new_mean)**2) / (total_N - 1)
                means[l] = new_mean
                Nl[l] = total_N

    # Final price is the sum of the means across all levels
    price = sum(means[:L+1])
    se = sqrt(sum(vars[l] / Nl[l] for l in range(L + 1)))
    total_cost = sum(Nl[:L+1] * costs[:L+1])

    print(f"\nFinal Price: {price:.6f} ± {se:.6f}")
    print(f"Total levels: {L+1}, Total cost: {total_cost:.0f}")
    return price, se, total_cost

def get_asian_reference_price(path_simulator, S0:float, K:float, r:float, sigma:float, T:float, n_steps:int=2048, n_paths:int=100000, seed:int=321211):
    """
    Calculates a high-resolution MC price to serve as a benchmark for weak error analysis

    :param path_simulator: Function for path simulation
    :param S0: Initial asset price
    :param K: Strike price
    :param r: Risk free rate
    :param sigma: Volatility parameter
    :param T: Time step
    :param n_steps: Number of MC steps
    :param n_paths: Number of MC paths
    :param seed: RNG seed

    """
    print("Calculating reference price for Asian option...")
    rng = np.random.default_rng(seed)
    dt = T / n_steps
    dW = rng.normal(0.0, sqrt(dt), size=(n_paths, n_steps))
    paths = path_simulator(S0, dW, dt, r, sigma)
    payoffs = payoff_asian_call(paths, K, dt)
    price = exp(-r * T) * np.mean(payoffs)
    se = np.std(payoffs, ddof=1) / sqrt(n_paths) * exp(-r * T)
    print(f"Reference Price: {price:.6f} ± {se:.6f}")
    return price

def weak_order_convergence_asian(scheme_name, path_simulator, ref_price:float, S0:float, K:float, r:float, sigma:float, T:float, Npaths:int=30000, seed:int=456):
    """
    Performs weak order analysis for an Asian option pricer
    :param scheme_name: Name of the scheme
    :param path_simulator: Function for path simulation
    :param S0: Initial asset price
    :param K: Strike price
    :param r: Risk free rate
    :param sigma: Volatility parameter
    :param T: Time step
    :param Npaths: Number of MC paths
    :param seed: RNG seed
    """
    print(f"\n=== Weak Order Convergence for Asian Option ({scheme_name} Scheme) ===")
    rng = np.random.default_rng(seed)
    results = []
    steps_list = [4, 8, 16, 32, 64, 128, 256]

    for n in steps_list:
        dt = T / n
        dW = rng.normal(0.0, sqrt(dt), size=(Npaths, n))
        paths = path_simulator(S0, dW, dt, r, sigma)
        P_approx = exp(-r * T) * payoff_asian_call(paths, K, dt)
        bias = np.mean(P_approx) - ref_price
        results.append((dt, abs(bias)))
        print(f"Steps: {n:3d}, dt: {dt:.4f}, bias: {abs(bias):.3e}")

    # Calculate convergence order (using middle points to avoid endpoint effects)
    dts = np.array([r[0] for r in results[1:-1]])
    biases = np.array([r[1] for r in results[1:-1]])
    if len(biases[biases > 0]) >= 3:
        log_dts = np.log(dts[biases > 0])
        log_biases = np.log(biases[biases > 0])
        slope, _ = np.polyfit(log_dts, log_biases, 1)
        print(f"Estimated weak order for {scheme_name}: {slope:.3f}")
    return slope

if __name__ == "__main__":
    # Standard parameters
    params = {'S0': 1.0, 'K': 1.0, 'r': 0.05, 'sigma': 0.20, 'T': 1.0}

    # Test both SRK versions
    schemes_to_test = {
        'Runge_Kutta': srk_scheme
    }

    for scheme_name, path_simulator in schemes_to_test.items():
        print("="*60)

        # Calculate reference price
        ref_price = get_asian_reference_price(path_simulator, **params)

        # Analyze weak convergence
        weak_order = weak_order_convergence_asian(scheme_name, path_simulator, ref_price, **params)

        # Run MLMC
        mlmc_price, mlmc_se, total_cost = mlmc_adaptive_asian(scheme_name, path_simulator, **params, eps=0.01, M=4)
